In [1]:
import folium
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import os

In [2]:
METADATA_PATH = '../data/metadata/train_val/trondheim/'

## Load Dataframes

In [35]:
DATA_PATH_QUERY = os.path.join(METADATA_PATH, 'query/raw.csv')
DATA_PATH_DATABASE = os.path.join(METADATA_PATH, 'database/raw.csv')

In [36]:
df = pd.read_csv(DATA_PATH_DATABASE)
dfq = pd.read_csv(DATA_PATH_QUERY)

## Inspect a City

In this code block, we will inspect the routes in the dataset. 

Different map types can be found on [Github](https://python-visualization.github.io/folium/modules.html)

In [88]:
MAP_TYPE = 'Stamen Terrain'

In [98]:
def Haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 * 10**3 #Radius of earth in meters.
    return c * r

In [99]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

In [94]:
def SplitCoordintesByFrame(coordinates, frames):
    previous_split = 0
    for i in range(1, len(frames)):
        if frames[i-1][0] + 1 != frames[i][0]:
            route_seq = coordinates[previous_split:i]
            folium.PolyLine(route_seq, weight=5.5).add_to(m)
            previous_split = i

In [103]:

# This is another way of generating the exact routes taken, but it is not super robust i.e. it seems to throw away some data. 

"""
DATA_PATH_DATABASE = os.path.join(METADATA_PATH, 'database/seq_info.csv')
df_seq = pd.read_csv(DATA_PATH_DATABASE)
df_full = pd.merge(df, df_seq[["key", "sequence_key", "frame_number"]], on='key')

for sequence_key in df_full["sequence_key"].unique():
    coordinates = df_full.loc[df_full["sequence_key"] == sequence_key][["lat", "lon", "frame_number"]]
    coordinates_list = coordinates[["lat", "lon"]].values.tolist()
    frames_list = coordinates[["frame_number"]].values.tolist()
    SplitCoordintesByFrame(coordinates_list, frames_list)
"""

'\nDATA_PATH_DATABASE = os.path.join(METADATA_PATH, \'database/seq_info.csv\')\ndf_seq = pd.read_csv(DATA_PATH_DATABASE)\ndf_full = pd.merge(df, df_seq[["key", "sequence_key", "frame_number"]], on=\'key\')\n\nfor sequence_key in df_full["sequence_key"].unique():\n    coordinates = df_full.loc[df_full["sequence_key"] == sequence_key][["lat", "lon", "frame_number"]]\n    coordinates_list = coordinates[["lat", "lon"]].values.tolist()\n    frames_list = coordinates[["frame_number"]].values.tolist()\n    SplitCoordintesByFrame(coordinates_list, frames_list)\n'

In [100]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5).add_to(m)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

In [101]:
m

### Inspect City by Season

In [88]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

In [89]:
SeasonsByMonth = [(month%12 + 3)//3 for month in range(1, 13)]
def DateToYear(Date):
    Month = int(Date.split('-')[1]) - 1
    return SeasonsByMonth[Month]

MonthToColor = {
    1 : 'Blue',
    2 : 'Yellow',
    3 : 'Green',
    4 : 'Red'
}


In [90]:
WinterGroup = folium.FeatureGroup(name='Winter')
SpringGroup = folium.FeatureGroup(name='Spring')
SummerGroup = folium.FeatureGroup(name='Summer')
FallGroup = folium.FeatureGroup(name='Fall')
MonthToGroup = [WinterGroup, SpringGroup, SummerGroup, FallGroup]

In [91]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            SeasonId = DateToYear(df['captured_at'][i])
            LineColor = MonthToColor[SeasonId]
            SeasonGroup = MonthToGroup[SeasonId - 1]
            folium.PolyLine(points, weight=5.5, color=LineColor).add_to(SeasonGroup)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

for FeatureGroup in MonthToGroup:
    FeatureGroup.add_to(m)

folium.map.LayerControl().add_to(m)

In [92]:
m

## Database and Query 

In [93]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

DBGroup = folium.FeatureGroup(name='Database')
QueryGroup = folium.FeatureGroup(name='Query')

In [94]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5, color='Red').add_to(DBGroup)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

point = 0,0
points = []
for i in range(0, len(dfq)):
    if Haversine(point[1],point[0], dfq['lon'][i], dfq['lat'][i]) > 25:
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5, color='Blue').add_to(QueryGroup) # 25 Meters (~82 feet)
        point = (dfq['lat'][i], dfq['lon'][i])
        points = [point]
    else:
        point = (dfq['lat'][i], dfq['lon'][i])
        points.append(point)

In [95]:
DBGroup.add_to(m)
QueryGroup.add_to(m)
folium.map.LayerControl().add_to(m)

m.save('DBAndQuery.html')
m

## Inspect City with Images

In [101]:
m = folium.Map(location=[df['lat'][0], df['lon'][0]], zoom_start=12, tiles=MAP_TYPE)

In [102]:
def PlotPointWithImage(Latitude, Longitude, ImageKey):
    html = "<img src='/Users/alexanderholstrup/git/VPRLossFunction/data/data/train_val/trondheim/database/images/{}.jpg'>".format(ImageKey)
    
    marker = folium.Marker(
        location=[Latitude, Longitude],
        radius=4,
        popup=html
    )
    marker.add_to(m)

In [103]:
for i in range(0, 800, 2):
    PlotPointWithImage(df['lat'][i], df['lon'][i], df['key'][i])

In [104]:
m.save('MapWithImages.html')